In [1]:
from matplotlib.pyplot import *
from sklearn import manifold
from numpy import *
from cube import *
import time
from openTSNE import TSNE
from openTSNE.callbacks import ErrorLogger

delete_duplicates = True

In [2]:
def val_from_colour(str):
    if str is 'W':
        return 0
    elif str is 'G':
        return 1
    elif str is 'R':
        return 2
    elif str is 'O':
        return 3
    elif str is 'Y':
        return 4
    elif str is 'B':
        return 5


def to_target_data(cube_data):
    # fill array with zeros
    # colors of a cubie described using one-hot: 1,0,0,0,0,0 might mean white=yes, red=no, blue=,...
    # there are 9 cubies per side
    # there a six sides per cube-setting
    # the number of cube-settings is len_array(cube_data[i])
    # there are several unique cubes being solved in several cube-settings

    output_array = []
    i = 0

    for cube in cube_data:
        for setting in cube:
            new_array = np.zeros(9 * 6 * 6)
            for side in setting:
                for row in side:
                    for val in row:
                        new_array[i + val_from_colour(val)] = 1
                        i += 6
            output_array.append(deepcopy(new_array))
            i = 0

    return output_array

In [6]:




# number of cubes to be solved using a simple algorithm with several checkpoints
# https://ruwix.com/the-rubiks-cube/how-to-solve-the-rubiks-cube-beginners-method/
loop_amt = 10
# if reuse_cube is True both algos are applied to the same random origin
reuse_cube = False
scramble_amt = 26
# at each position the solution process of a unique cube will be stored - all settings from random to solved
data = []
moves = []
# same thing for the indices of the cubes that represent check points
check_points = []
algotype = []
differences = []

for i in range(loop_amt):
    print("scrambling "+str(i)+"...")
    scramble(scramble_amt)
    mvs = get_scramble()
    print("solving "+str(i)+"...")
    d, cp, diff, solution = solve_simple()

    print("moves...")
    print(solution)
    data.append(d)
    moves.append(solution.split())
    check_points.append(cp)
    algotype.append("0")
    differences.append(diff)
    print("amt cps", len(cp))
    print("len moves", len(solution.split()))

    print("scrambling " + str(i) + "...")
    if reuse_cube:
        scramble(mvs)
    else:
        scramble(scramble_amt)
    print("solving " + str(i) + "...")
    d, cp, diff, solution = solve()
    data.append(d)
    moves.append(solution.split())
    check_points.append(cp)
    algotype.append("1")
    differences.append(diff)
    print("amt cps", len(cp))
    print("len moves", len(solution.split()))

for j in range(2 * loop_amt):
    print("checkpoints", check_points[j])
    print("data len", len(data[j]))
    data[j]
    sol_len = len(data)

cube_data = data

# solution of cube 1 lies in cube_data between the indices sol_len[0] and sol_len[1]
# solution of cube 2 lies in cube_data between the indices sol_len[1] and sol_len[2]
# ...
sol_len = [0]
for i in range(len(data)):
    print("len of cube:", len(data[i]))
    sol_len.append(sol_len[i] + len(data[i]))


# turning [[[['R', 'O', 'O'], ...] data structure into one-hot encoded data structure
used_data = to_target_data(cube_data)


print(len(used_data))

if delete_duplicates:
    used_data, inds, cs = np.unique(used_data, axis=0, return_inverse=True, return_counts=True)
    
print(len(used_data))

# calculating t-sne on datastructure holding all solved cubes with all steps in between and their one-hot encoded colors
print("calculating t-SNE for "+str(len(data))+" cubes...")
start_time = time.time()
X_tsne = manifold.TSNE(learning_rate=100, perplexity=50).fit_transform(used_data)
print("elapsed time: %.3fs" % (time.time()-start_time))

print(len(X_tsne))

if delete_duplicates:
    X_tsne = X_tsne[inds]
    
print(len(X_tsne))



scrambling 0...
solving 0...
moves...
Di F2 D2 Ui F R U Ri U2 B2 U2 F2 U2 R Ui Ri U2 R Ui R2 U2 R2 Bi Ri B Ui Li U L U Li U L2 U Li U R U Ri U R U2 Ri U Fi U B Ui F U Bi Ui Fi U B Ui F U Bi Ui Bi Di B D Bi Di B D Ui Bi Di B D Bi Di B D Bi Di B D Bi Di B D Ui Bi Di B D Bi Di B D Ui Bi Di B D Bi Di B D Bi Di B D Bi Di B D Ui
amt cps 5
len moves 112
scrambling 0...
solving 0...
amt cps 5
len moves 100
scrambling 1...
solving 1...
moves...
F R U Ri F2 Di Fi R U Ri F2 Di F R U Ri F2 Di Ui F2 D L2 Ui F2 U L2 Ui Ri U2 R2 U Ri U B2 U2 Bi Ui B Ui B2 Li U L Ui L Ui Li Ui Bi Ui B F2 U F2 U F2 U2 F2 U F R U Ri Ui Fi L U Li U L U2 Li U Ri U L Ui R U Li Ui Ri U L Ui R U Li Ui Li Di L D Li Di L D Ui Li Di L D Li Di L D Li Di L D Li Di L D Ui Li Di L D Li Di L D Li Di L D Li Di L D Ui Li Di L D Li Di L D Ui
amt cps 5
len moves 141
scrambling 1...
solving 1...
amt cps 5
len moves 112
scrambling 2...
solving 2...
moves...
F Di U F R U Ri F2 Di F2 R U Ri F2 Di U F R U Ri F2 D2 B2 Ui L2 U B2 Ri U R Ui R2 

In [10]:
file = open("csv\\cube.csv", "w")
header = "x,y,line,cp,algo,changes"
sides = ["up", "front", "right", "left", "down", "back"]
for side in sides:
    for i in range(0,3):
        for j in range(0,3):
            header += ","+side+str(i)+str(j)
            
            
names = ['beginner','fridrich']
header += "\n"
# resulting header:
# x,y,cubeNum,up00,up01,up02,up10,up11,up12,up20,up21,up22,front00,front01,...
# x and y being coordinates of t-sne plot and up00 holding a letter for the color of the "up"-side top left cubie...
file.write(header)

cube_iterator = 0
i = 0
cp_i = 0
setting_i = 0
print(check_points)
for cube in cube_data:
    print(len(cube))
    print(len(moves[cube_iterator]))
    for si, setting in enumerate(cube):
        if check_points[cube_iterator][cp_i] == setting_i:
            cp_str = "checkpoint"
            cp_i += 1
        else:
            cp_str = "intermediate"
        
        if si == 0:
            cp_str = 'start'
        if si == len(cube) - 1:
            cp_str = 'end'
        
        if si != 0:
            mov = moves[cube_iterator][si - 1]
        else:
            mov = ''
    
        
        file.write("%s,%s,%s,%s,%s,%s" %(X_tsne[i][0], X_tsne[i][1], cube_iterator, cp_str, names[int(algotype[cube_iterator])], mov))
        for side in setting:
                for row in side:
                    for cubie in row:
                        file.write(",%s" %cubie)
        file.write("\n")
        i += 1
        setting_i += 1
    cube_iterator += 1
    setting_i = 0
    cp_i = 0
    
file.close()

# plotting using matplotlib
# figure(figsize=(10, 5))
# for i in range(1, len(sol_len)):
#    plot(X_tsne[sol_len[i-1] + i-1:sol_len[i] + i, 0], X_tsne[sol_len[i-1] + i-1:sol_len[i] + i, 1], alpha=0.7, zorder=0)
#    scatter(X_tsne[sol_len[i - 1] + i - 1, 0], X_tsne[sol_len[i - 1] + i - 1, 1], c='m', zorder=1)
#    scatter(X_tsne[sol_len[i] + i - 1, 0], X_tsne[sol_len[i] + i - 1, 1], c='y', zorder=1)


# show()

[[0, 12, 35, 44, 112], [0, 26, 65, 73, 100], [0, 26, 58, 65, 141], [0, 32, 62, 70, 112], [0, 27, 64, 78, 147], [0, 22, 61, 68, 76, 84, 106], [0, 27, 62, 63, 105], [0, 28, 58, 72, 80, 88, 97], [0, 23, 54, 64, 65, 107], [0, 26, 74, 82, 95], [0, 21, 60, 61, 128], [0, 30, 67, 75, 83, 96], [0, 18, 63, 72, 114], [0, 26, 58, 69, 98], [0, 29, 66, 116], [0, 15, 54, 61, 69, 86], [0, 63, 88, 114], [0, 10, 53, 70], [0, 27, 61, 75, 126], [0, 21, 65, 73, 81, 104]]
113
112
101
100
142
141
113
112
148
147
107
106
106
105
98
97
108
107
96
95
129
128
97
96
115
114
99
98
117
116
87
86
115
114
71
70
127
126
105
104


In [ ]:
test